![banner](https://github.com/paulguz261/MIAD_20242_proyecto_despliegue_aplicaciones/blob/main/docs/images/despliegue.png)

# Importación de librerías  

In [27]:
import pandas as pd
import numpy as np
from IPython.display import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import metrics
import optuna

import warnings
warnings.filterwarnings('ignore')

# Cargar datos

In [2]:
data = pd.read_csv("../data/clean/personas_limpio.csv")
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1


In [3]:
# Dado que las variables binarias quedaron como númericas, se procede a convertirlas de nuevo
def convert_to_categorical(df, columns):
    """
    Convierte las columnas especificadas de un DataFrame a tipo categórico.

    Parámetros:
    df (pd.DataFrame): El DataFrame en el que se realizarán las conversiones.
    columns (list): Lista de nombres de columnas a convertir a tipo categórico.

    Retorna:
    pd.DataFrame: El DataFrame con las columnas especificadas convertidas a tipo categórico.
    """
    for column in columns:
        if column in df.columns:
            df[column] = pd.Categorical(df[column])
        else:
            print(f"Columna '{column}' no encontrada en el DataFrame.")
    return df

categorical_columns = ['hypertension', 'heart_disease','stroke']
data = convert_to_categorical(data, categorical_columns)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   gender             5110 non-null   object  
 1   age                5110 non-null   float64 
 2   hypertension       5110 non-null   category
 3   heart_disease      5110 non-null   category
 4   ever_married       5110 non-null   object  
 5   work_type          5110 non-null   object  
 6   Residence_type     5110 non-null   object  
 7   avg_glucose_level  5110 non-null   float64 
 8   bmi                5110 non-null   float64 
 9   smoking_status     5110 non-null   object  
 10  stroke             5110 non-null   category
dtypes: category(3), float64(3), object(5)
memory usage: 334.8+ KB


El conjunto de datos `personas_limpio`contiene 5.110 registros y 11 columnas resultado de la limpieza de datos aplicada. Las variables categóricas son `gender`, `ever_married`, `work_type`, `resident_type`y `smoking_status`, variables que indican factores importantes a la hora de predecir un ACV. Además, las variables númericas tales `age`, `avg_glucose_level` y `bmi` indican edad y estado de salud de la persona. Y las variables binarias `hypertension` y `heart_disase`, las cuales indican el padecimiento de cierta patologia. Finalmente, la variable a predecir `stroke` indica la presencia de ACV en la persona.

# Procesamiento de los datos

En este apartado se definen las variables predictoras y a predecir, se codifican las variables categoricas aplicando One-Hot Encoding para transformarlas en variables númericas y se aplica reducción de dimensionalidad con PCA. Además, se divide el conjunto de datos en entrenamiento y prueba para la construcción y prueba de los modelos.

In [20]:
# Definición de las variables predictoras y a predecir
X = data.drop(columns=['stroke'])  # Eliminar variable a predecir
y = data['stroke']

In [21]:
# Conversión de variables categoricas a numericas con One-Hot Encoding
data_dummies = pd.get_dummies(X,columns=X.select_dtypes(include=['object','category']).columns.to_list())

# Estandarización de las variables
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_dummies)

In [22]:
# Reducción de dimensionalidad PCA
pca = PCA(n_components=0.95)  # COnservar el 95% de la varianza
data_pca = pca.fit_transform(data_scaled)

print(f"Dimensiones antes de PCA: {data_scaled.shape}")
print(f"Dimensiones después de PCA: {data_pca.shape}")

Dimensiones antes de PCA: (5110, 23)
Dimensiones después de PCA: (5110, 14)


In [23]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_pca, y, test_size=0.33, random_state=40)

# Ajuste hiperparametros

En esta sección se realizara el ajuste de hiperparametros en cada uno de los modelos de clasificación probados, los mejores hiperparametros seran almacenados en MLFlow.

In [ ]:
# Modelo básico
clf = XGBClassifier()
# Entrenamiento (fit) y desempeño del modelo XGBClassifier
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test.values), metrics.accuracy_score(y_pred, y_test.values)

(0.06060606060606061, 0.944872554831061)

In [29]:
def objective(trial):
    # Hiperparametros a probar
    param = {
        'verbosity': 0,
        'objective': 'binary:logistic',  # Clasificación binaria
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 8),
        'reg_lambda' : trial.suggest_int('reg_lambda',1, 10),
        'reg_alpha' : trial.suggest_int('reg_alpha',1, 10)
    }

    # Create and fit the model
    clf = XGBClassifier(**param)
    clf.fit(X_train, y_train)

    # Predict and calculate RMSE
    preds = clf.predict(X_test)
    acc = metrics.accuracy_score(y_pred, y_test.values)
    f1_score = metrics.f1_score(y_pred, y_test.values)

    return f1_score

In [30]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-11-07 23:53:27,786] A new study created in memory with name: no-name-cd02dce6-207b-4b72-a016-c68cb5d57efc
[I 2024-11-07 23:53:28,401] Trial 0 finished with value: 0.06060606060606061 and parameters: {'n_estimators': 281, 'max_depth': 10, 'learning_rate': 0.04262930661938335, 'subsample': 0.7198398803103419, 'colsample_bytree': 0.8759758796282655, 'min_child_weight': 5, 'reg_lambda': 7, 'reg_alpha': 6}. Best is trial 0 with value: 0.06060606060606061.
[I 2024-11-07 23:53:28,793] Trial 1 finished with value: 0.06060606060606061 and parameters: {'n_estimators': 123, 'max_depth': 9, 'learning_rate': 0.08283391197493284, 'subsample': 0.9648964371451663, 'colsample_bytree': 0.9077475703788395, 'min_child_weight': 6, 'reg_lambda': 7, 'reg_alpha': 1}. Best is trial 0 with value: 0.06060606060606061.
[I 2024-11-07 23:53:29,263] Trial 2 finished with value: 0.06060606060606061 and parameters: {'n_estimators': 249, 'max_depth': 7, 'learning_rate': 0.07073266615772653, 'subsample': 0.99681